# **Import Library**

In [ ]:
# Import library
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# **Load and Read Dataset**

In [ ]:
# Import dataset items
df_items = pd.read_csv('items.csv', sep='|')
df_items

,itemID,title,author,publisher,main topic,subtopics
0,21310,Princess Poppy: The Big Mix Up,Janey Louise Jones,Penguin Random House Children's UK,YFB,[5AH]
1,73018,Einfach zeichnen! Step by Step,Wiebke Krabbe,Schwager und Steinlein,AGZ,"[5AJ,AGZ,WFA,YBG,YBL,YNA,YPA]"
2,19194,Red Queen 1,Victoria Aveyard,Orion Publishing Group,YFH,"[5AP,FBA]"
3,40250,Meine Kindergarten-Freunde (Pirat),NaN,Ars Edition GmbH,YB,"[5AC,5AD,YBG,YBL,YF]"
4,46107,Mein großes Schablonen-Buch - Wilde Tiere,Elizabeth Golding,Edition Michael Fischer,WFTM,"[WD,WFTM,YBG,YBL,YBLD,YBLN1]"
...,...,...,...,...,...,...
78025,37678,Timeless Fairy Tales,"Brothers Grimm, Marie-Catherine Baroness D'Aulnoy",MEDIAMORPHOSIS,YFA,[YFJ]
78026,68688,Demon Games,A. Witt Timothy a. Witt,iUniverse,FMB,[]
78027,57291,Lori and the Lion's Den,A. M. Glass,Xlibris,YFU,[]
78028,78130,The Everywhere Armchair,Ersila Bee,ELOQUENT BOOKS,YFC,"[YFG,YFH]"


In [ ]:
# Import dataset transactions
df_transactions = pd.read_csv('transactions.csv', sep='|')
df_transactions

,sessionID,itemID,click,basket,order
0,0,21310,1,0,0
1,1,73018,1,0,0
2,2,19194,1,0,0
3,3,40250,1,0,0
4,4,46107,1,0,0
...,...,...,...,...,...
365138,279351,70183,1,0,0
365139,279352,39716,1,0,0
365140,279353,35260,1,0,0
365141,279353,18805,4,0,0


# **Preprocessing**

In [ ]:
# Cek missing values
df_transactions.isna().sum()

sessionID    0
itemID       0
click        0
basket       0
order        0
dtype: int64

In [ ]:
df_items.isna().sum()

itemID           0
title            0
author        3240
publisher        9
main topic     258
subtopics        1
dtype: int64

In [ ]:
# Drop missing values
df_items = df_items.dropna()
df_items = df_items.reset_index(drop=True)
df_items.isna().sum()

itemID        0
title         0
author        0
publisher     0
main topic    0
subtopics     0
dtype: int64

# **Merge Dataset**

In [ ]:
# Merge dataset items dan transactions
df_merge = df_items.merge(df_transactions, left_on='itemID', right_on='itemID')
df_merge

,itemID,title,author,publisher,main topic,subtopics,sessionID,click,basket,order
0,21310,Princess Poppy: The Big Mix Up,Janey Louise Jones,Penguin Random House Children's UK,YFB,[5AH],0,1,0,0
1,21310,Princess Poppy: The Big Mix Up,Janey Louise Jones,Penguin Random House Children's UK,YFB,[5AH],89272,1,0,0
2,21310,Princess Poppy: The Big Mix Up,Janey Louise Jones,Penguin Random House Children's UK,YFB,[5AH],183856,1,0,0
3,73018,Einfach zeichnen! Step by Step,Wiebke Krabbe,Schwager und Steinlein,AGZ,"[5AJ,AGZ,WFA,YBG,YBL,YNA,YPA]",1,1,0,0
4,73018,Einfach zeichnen! Step by Step,Wiebke Krabbe,Schwager und Steinlein,AGZ,"[5AJ,AGZ,WFA,YBG,YBL,YNA,YPA]",771,1,0,0
...,...,...,...,...,...,...,...,...,...,...
323086,8686,Beasts of Prey,Ayana Gray,Penguin LCC US,YFH,"[1H,YFB,YFC,YNM]",279161,1,0,0
323087,21264,Soon,Morris Gleitzman,Penguin Random House Children's UK,YFN,[5AL],279176,1,0,0
323088,56041,Tesla,Mark Lingane,Insync Holdings Pty Ltd,FLM,[],279248,1,0,0
323089,67832,Hexenherz. Glühender Hass,Monika Loerchner,Acabus Verlag,FMH,"[1D,FMH,FMR]",279343,2,0,0


In [ ]:
# Cek info data
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 323091 entries, 0 to 323090
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   itemID      323091 non-null  int64 
 1   title       323091 non-null  object
 2   author      323091 non-null  object
 3   publisher   323091 non-null  object
 4   main topic  323091 non-null  object
 5   subtopics   323091 non-null  object
 6   sessionID   323091 non-null  int64 
 7   click       323091 non-null  int64 
 8   basket      323091 non-null  int64 
 9   order       323091 non-null  int64 
dtypes: int64(5), object(5)
memory usage: 27.1+ MB


In [ ]:
# Karena itemID akan dibuat menjadi list maka tipenya diubah menjadi string
df_merge['itemID'] = df_merge['itemID'].astype(str)

In [ ]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 323091 entries, 0 to 323090
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   itemID      323091 non-null  object
 1   title       323091 non-null  object
 2   author      323091 non-null  object
 3   publisher   323091 non-null  object
 4   main topic  323091 non-null  object
 5   subtopics   323091 non-null  object
 6   sessionID   323091 non-null  int64 
 7   click       323091 non-null  int64 
 8   basket      323091 non-null  int64 
 9   order       323091 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 27.1+ MB


In [ ]:
# Mengelompokkan string itemID sesuai dengan sessionID yang sama
df_join = df_merge.groupby(['sessionID'])['itemID'].apply(','.join).reset_index()
df_temp = df_join

# Column yang akan di merge untuk dibuat list
x = ['title','author', 'publisher', 'main topic', 'subtopics']

for i in x:
  df_join = df_merge.groupby(['sessionID'])[i].apply(','.join).reset_index()
  df_temp = df_temp.merge(df_join, left_on = 'sessionID', right_on='sessionID')

# Menjumlahkan nilai pada column bertipe int
df_join = df_merge.groupby(['sessionID']).sum()
df_temp = df_temp.merge(df_join, left_on='sessionID', right_on='sessionID')
df_temp.tail()

,sessionID,itemID,title,author,publisher,main topic,subtopics,click,basket,order
242768,279349,3051,Elfenmacht,Bernhard Hennen,Heyne Verlag,FMB,[],1,0,0
242769,279350,"13996,23084,50891,73084","Wonder,Dear Evan Hansen,Dear Evan Hansen,Every...","R. J. Palacio,Val Emmich, Justin Paul, Steven ...","Random House Children's,Penguin Books Ltd (UK)...","YXQF,YXD,YXLD2,YXQF","[5AK,YFS,YP,YXD,YXF],[5AN,YFB,YXG],[5AQ,5LF,1K...",4,0,0
242770,279351,70183,House of Earth and Blood,Sarah J. Maas,BLOOMSBURY,FMR,"[FMX,FRT]",1,0,0
242771,279352,39716,Das Farbenmonster,Anna Llenas,Velber Verlag,YBLD,"[5AC,YBLD,YNN,YXE]",1,0,0
242772,279353,"18805,35260","Und plötzlich war Frau Honig da,Die Wilden Hüh...","Sabine Bohlmann,Cornelia Funke","Carlsen Verlag GmbH,Oetinger Taschenbuch GmbH","YFN,YFC","[5AJ,YFQ],[5AK,5AL,5JA]",5,0,0


In [ ]:
df_temp.shape

(242773, 10)

# **Collaborative Filtering**

**User Item**

In [ ]:
def make_user_item_matrix(data):
    user_item_matrix = data.pivot_table(
        index = 'sessionID', 
        columns = 'itemID', 
        values = 'click', 
        aggfunc = 'sum'
    )
    # Ketika pelanggan klik buku maka session nilai nya 1 dan sebaliknya
    user_item_matrix = user_item_matrix.applymap(lambda x: 1 if x > 0 else 0)
    return user_item_matrix

In [ ]:
# Mengambil jumlah data yang akan dijadikan sample
df_sample = df_temp[:15000]

In [ ]:
# Membuat item matrix dari df_sample
user_item_matrix = make_user_item_matrix(df_sample)
user_item_matrix

itemID,1,10004,10031,10036,10043,10082,10091,"10091,19894,71701",10095,10120,10124,10148,1017,10172,"10172,23082,31927,63505",10190,10195,"10195,28502","10195,77474",10202,10213,10258,10269,10284,10305,"10305,26026","10305,42152",10338,10358,10381,10420,10438,10495,10522,"10522,74059,20276,65132",10537,10541,10542,10550,1059,...,9602,9607,961,"961,49479,14977",9658,966,9671,9690,9720,9736,9740,"9740,69803",9766,9771,9789,"9789,58061,32854,73643",9795,"9795,30237","9795,50356,39253,60780,43203,64507","9795,55334,56860",9808,9816,9849,9852,9856,9865,"9865,55190",9868,9870,"9870,12224","9927,26154",994,9960,9971,9972,"9972,44954,57212,23622",9976,9983,9984,9990
sessionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17301,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17302,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17303,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Item to Item Similarity Matrix**

In [ ]:
# Menghitung similarity (kemiripan) antar item
item_matrix = pd.DataFrame(cosine_similarity(user_item_matrix.T))
item_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,6943,6944,6945,6946,6947,6948,6949,6950,6951,6952,6953,6954,6955,6956,6957,6958,6959,6960,6961,6962,6963,6964,6965,6966,6967,6968,6969,6970,6971,6972,6973,6974,6975,6976,6977,6978,6979,6980,6981,6982
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
# Penamaan baris dan kolom
item_matrix.columns = user_item_matrix.T.index # kolom yang diambil dari index user item matrix
item_matrix['itemID'] = user_item_matrix.T.index # baris yang diambil dari index user item matrix
item_matrix = item_matrix.set_index('itemID') # di set indexnya
item_matrix.head()

itemID,1,10004,10031,10036,10043,10082,10091,"10091,19894,71701",10095,10120,10124,10148,1017,10172,"10172,23082,31927,63505",10190,10195,"10195,28502","10195,77474",10202,10213,10258,10269,10284,10305,"10305,26026","10305,42152",10338,10358,10381,10420,10438,10495,10522,"10522,74059,20276,65132",10537,10541,10542,10550,1059,...,9602,9607,961,"961,49479,14977",9658,966,9671,9690,9720,9736,9740,"9740,69803",9766,9771,9789,"9789,58061,32854,73643",9795,"9795,30237","9795,50356,39253,60780,43203,64507","9795,55334,56860",9808,9816,9849,9852,9856,9865,"9865,55190",9868,9870,"9870,12224","9927,26154",994,9960,9971,9972,"9972,44954,57212,23622",9976,9983,9984,9990
itemID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10004,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10031,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10036,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10043,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Menampilkan 5 rekomendasi itemID teratas berbentuk list berdasarkan itemID 9960
top5_itemID = list(item_matrix.loc['9960'].sort_values(ascending=False).iloc[:5].index)
top5_itemID

['9960', '9990', '33338', '33211', '33235']

In [ ]:
# Menampilkan 5 rekomendasi teratas berdasarkan itemID 9960
df_sample.loc[ 
              df_sample['itemID'].isin(top5_itemID), ['itemID', 'title',	'author',	'publisher', 'main topic']
              ].drop_duplicates().set_index('itemID').loc[top5_itemID]

,title,author,publisher,main topic
itemID,,,,
9960,Jaghatai Khan: Warhawk of Chogoris,Chris Wraight,GAMES WORKSHOP,FLS
9990,Spider-Man: Ewige Jugend: Ein Roman aus dem Ma...,Stefan Petrucha,Panini Verlags GmbH,FYT
33338,The Dragon's Eye,Dugald Steer,Templar Publishing,YFH
33211,Planetenleuchten,Sarah Scheumer,Books on Demand,FLS
33235,Eines Menschen Flügel,Andreas Eschbach,Lübbe,FL


In [ ]:
# Export 5 rekomendasi itemID teratas berdasarkan itemID 9960
top5 = pd.DataFrame(top5_itemID, columns=['itemID'])
top5 = top5.set_index('itemID')
top5.to_csv('evaluation.csv')
df_evaluation = pd.read_csv('evaluation.csv')
df_evaluation

,itemID
0,9960
1,9990
2,33338
3,33211
4,33235
